# Implementing a Simple Markov Chain Model for Commentary Generation

In [ ]:
import random
import re
import nltk
from collections import defaultdict, Counter
import pandas as pd
import numpy as np

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Sample commentary corpus for demonstration
commentary_corpus = [
    "James drives to the basket, lays it up and in! Two points for the Lakers.",
    "Curry with the ball, steps back, shoots the three... BANG! Nothing but net!",
    "Durant crosses over, pulls up from mid-range, and it's good! Smooth jumper.",
    "James passes to Davis, who dunks it with authority! What a play!",
    "Curry dribbles past the defender, stops at the line, shoots the three... It's good!",
    "Harden with the step-back three... and he connects! Three points for the Rockets.",
    "Antetokounmpo drives, spins, and slams it home! What a move by the Greek Freak!",
    "James with the no-look pass to Davis for the alley-oop slam!",
    "Curry finds Green, back to Curry, shoots the three... BANG! From way downtown!",
    "Durant with the crossover, drives, and finishes with the left hand! Beautiful move."
]

In [ ]:
class MarkovChainGenerator:
    def __init__(self, corpus, order=2):
        self.order = order
        self.corpus = corpus
        self.model = defaultdict(Counter)
        self.build_model()

    def preprocess_text(self, text):
        text = f"{'__START__ ' * self.order}{text} __END__"
        return text

    def build_model(self):
        for text in self.corpus:
            text = self.preprocess_text(text)
            words = text.split()
            for i in range(len(words) - self.order):
                state = tuple(words[i:i + self.order])
                next_word = words[i + self.order]
                self.model[state][next_word] += 1

    def generate_text(self, max_length=50):
        current = ('__START__',) * self.order
        result = []
        for _ in range(max_length):
            if current not in self.model or '__END__' in current:
                break
            possibilities = self.model[current]
            choices, weights = zip(*possibilities.items())
            next_word = random.choices(choices, weights=weights, k=1)[0]
            if next_word == '__END__':
                break
            result.append(next_word)
            current = current[1:] + (next_word,)
        return ' '.join(result)

    def generate_multiple(self, count=5, max_length=50):
        return [self.generate_text(max_length) for _ in range(count)]

In [ ]:
from collections import defaultdict, Counter

class MarkovChainGenerator:
    def __init__(self, corpus, order=1):
        self.order = order
        self.corpus = corpus
        self.model = defaultdict(Counter)

    def preprocess_text(self, text):
        """Each sentence is padded with __START__ tokens at the beginning (equal to order) and one __END__ token at the end."""
        text = f"{'__START__ ' * self.order}{text} __END__"
        return text

# Example usage
corpus = ["He shoots and scores"]  # Example sentence
order = 2                         # Try changing to 1 or 3 to see the difference
markov_model = MarkovChainGenerator(corpus, order)

# Apply preprocess_text and print result
for sentence in corpus:
    processed = markov_model.preprocess_text(sentence)
    print("Original Sentence: ", sentence)
    print("Processed Sentence:", processed)

Original Sentence:  He shoots and scores
Processed Sentence: __START__ __START__ He shoots and scores __END__


| State                      | Next Word |
| -------------------------- | --------- |
| (`__START__`, `__START__`) | 'He'      |
| (`__START__`, 'He')        | 'shoots'  |
| ('He', 'shoots')           | 'and'     |
| ('shoots', 'and')          | 'scores'  |
| ('and', 'scores')          | `__END__` |


In [ ]:
# Run and generate
markov_model_order1 = MarkovChainGenerator(commentary_corpus, order=1)
print("\nGenerated Basketball Commentary Examples (1st-order Markov):")
for i, text in enumerate(markov_model_order1.generate_multiple(5), 1):
    print(f"{i}. {text}")


Generated Basketball Commentary Examples (1st-order Markov):
1. James passes to Curry, shoots the ball, steps back, shoots the no-look pass to the left hand! Beautiful move.
2. Curry dribbles past the three... BANG! From way downtown!
3. James drives to Davis for the defender, stops at the crossover, drives, and in! Two points for the ball, steps back, shoots the Rockets.
4. Curry with authority! What a move by the step-back three... BANG! Nothing but net!
5. Antetokounmpo drives, and slams it with the Rockets.


**🔍 Order 1 (First-Order Markov Model):**
The model predicts the next word based only on the immediately previous word.

Example:

Suppose your corpus contains:

"He shoots and he scores"

After training, the model learns:

"He" → ["shoots", "scores"]

"shoots" → ["and"]

"and" → ["he"]

"scores" → [END]

When generating text, the model decides the next word by only considering the last word generated.

In [ ]:
# Run and generate
markov_model_order1 = MarkovChainGenerator(commentary_corpus, order=2)
print("\nGenerated Basketball Commentary Examples (1st-order Markov):")
for i, text in enumerate(markov_model_order1.generate_multiple(5), 1):
    print(f"{i}. {text}")


Generated Basketball Commentary Examples (1st-order Markov):
1. Durant crosses over, pulls up from mid-range, and it's good! Smooth jumper.
2. Curry finds Green, back to Curry, shoots the three... BANG! Nothing but net!
3. Curry dribbles past the defender, stops at the line, shoots the three... BANG! From way downtown!
4. James with the ball, steps back, shoots the three... BANG! Nothing but net!
5. Curry finds Green, back to Curry, shoots the three... BANG! Nothing but net!


**🔍 Order 2 (Second-Order Markov Model):**
The model predicts the next word based on the previous two words.

Example:

From the same corpus:

"He shoots and he scores"

The model learns:

("He", "shoots") → ["and"]

("shoots", "and") → ["he"]

("and", "he") → ["scores"]

("he", "scores") → [END]

So, when generating text, the model picks the next word based on the last two words generated.

# Creating Templates for Different Game Scenarios

In [ ]:
import random
import json

In [ ]:
class TemplateGenerator:
    def __init__(self):
        """Initialize the template-based generator with predefined templates."""
        # Game events templates
        self.scoring_templates = [
            "{player} {scoring_action} {scoring_descriptor}! {points} for {team}.",
            "{player} {movement_action} and {scoring_action}! {celebration}.",
            "{player} with the {shot_type}... {result_exclamation} {points} on the board for {team}.",
            "{assist_phrase} {player} {scoring_action}! {excitement_phrase}",
            "What a {shot_descriptor} by {player}! {scoring_action} to {score_effect} for {team}." # Here---- What a move by Kevin Durant! nails it to pull ahead for Nets.
        ]

        # Player actions
        self.player_actions = {
            "movement_action": [
                "drives to the basket", "cuts inside", "finds space", "breaks free",
                "spins past the defender", "crosses over", "steps back", "gets past his man"
            ],
            "scoring_action": [
                "scores", "puts it in", "gets the basket", "sinks it", "nails it",
                "makes it count", "converts", "finishes"
            ],
            "scoring_descriptor": [
                "with ease", "with a beautiful shot", "despite the pressure",
                "with perfect technique", "under heavy defense", "with precision"
            ],
            "shot_type": [
                "three-pointer", "jump shot", "layup", "dunk", "fadeaway",
                "hook shot", "floater", "mid-range jumper"
            ],
            "result_exclamation": [
                "It's good!", "Count it!", "Nothing but net!", "It drops!",
                "Bang!", "Swish!", "Yes!"
            ],
            "celebration": [
                "The crowd goes wild", "His teammates love it", "What a moment",
                "That's what they needed", "Spectacular play"
            ],
            "assist_phrase": [
                "Great pass from {assisting_player} to",
                "{assisting_player} finds",
                "Brilliant assist by {assisting_player} and",
                "Perfect feed from {assisting_player} to"
            ],
            "excitement_phrase": [
                "What a play!", "Brilliant execution!", "That's how it's done!",
                "Textbook basketball!", "The fans are loving it!"
            ],
            "shot_descriptor": [
                "move", "finish", "shot", "play", "execution", "technique"
            ],
            "score_effect": [
                "extend the lead", "narrow the gap", "tie the game",
                "pull ahead", "put points on the board", "add to their total"
            ]
        }

        # Game state templates
        self.game_state_templates = [
            "{time_remaining} in the {period}, {team} {lead_status} {score_difference}.",
            "We're {time_descriptor} with {team} {lead_verb} {score}.",
            "{team} {momentum_phrase} as we {period_stage}.",
            "The score is {score} with {time_remaining} to play in the {period}."
        ]

        # Game state elements
        self.game_state_elements = {
            "time_remaining": [
                "2 minutes left", "45 seconds remaining", "just under a minute to go",
                "3:30 remaining", "half a minute left", "10 seconds on the clock"
            ],
            "period": [
                "first quarter", "second quarter", "third quarter", "fourth quarter",
                "first half", "second half", "game", "overtime"
            ],
            "lead_status": [
                "leading by", "up by", "ahead by", "in front by", "with a lead of"
            ],
            "lead_verb": [
                "leading", "ahead", "in front", "up", "on top"
            ],
            "time_descriptor": [
                "late in the game", "early in the first quarter", "midway through the third",
                "at the start of the fourth", "in the final minutes", "at a crucial stage"
            ],
            "momentum_phrase": [
                "building momentum", "making a comeback", "holding steady",
                "dominating this quarter", "struggling to score", "playing excellent defense"
            ],
            "period_stage": [
                "approach halftime", "head toward the final quarter",
                "near the end of regulation", "enter the closing minutes"
            ]
        }

    def fill_template(self, template, data):
        """Fill a template with provided data."""
        result = template
        for key, value in data.items():
            if isinstance(value, list):
                value = random.choice(value)
            if key in result:
                result = result.replace("{" + key + "}", value)
        return result

    def generate_scoring_commentary(self, player, team, points="two points", assisting_player=None):
        """Generate commentary for a scoring event."""
        template = random.choice(self.scoring_templates)

        data = {
            "player": player,
            "team": team,
            "points": points,
            "assisting_player": assisting_player if assisting_player else "teammate"
        }

        # Add random elements from action dictionaries
        for key, options in self.player_actions.items():
            data[key] = random.choice(options)

        return self.fill_template(template, data)

    def generate_game_state_commentary(self, team="Team A", score="67-65", time_remaining=None, period=None):
        """Generate commentary describing the game state."""
        template = random.choice(self.game_state_templates)

        data = {
            "team": team,
            "score": score,
            "score_difference": f"{random.randint(1, 15)} points"
        }

        # Add random elements if not specified
        if time_remaining is None:
            data["time_remaining"] = random.choice(self.game_state_elements["time_remaining"])
        else:
            data["time_remaining"] = time_remaining

        if period is None:
            data["period"] = random.choice(self.game_state_elements["period"])
        else:
            data["period"] = period

        # Add random elements from state dictionaries
        for key, options in self.game_state_elements.items():
            if key not in data:
                data[key] = random.choice(options)

        return self.fill_template(template, data)

In [ ]:
# Demonstrate the template generator
template_gen = TemplateGenerator()

print("Scoring Commentary Examples:")
players = ["LeBron James", "Stephen Curry", "Kevin Durant", "Giannis Antetokounmpo"]
teams = ["Lakers", "Warriors", "Nets", "Bucks"]
points_options = ["two points", "three points", "another basket"]

for i in range(5):
    player = random.choice(players)
    team = teams[players.index(player)]  # Match player with their team
    points = random.choice(points_options)
    assisting_player = random.choice([p for p in players if p != player])

    commentary = template_gen.generate_scoring_commentary(
        player=player,
        team=team,
        points=points,
        assisting_player=assisting_player
    )
    print(f"{i+1}. {commentary}")

print("\nGame State Commentary Examples:")
for i in range(3):
    team = random.choice(teams)
    score = f"{random.randint(50, 100)}-{random.randint(50, 100)}"

    commentary = template_gen.generate_game_state_commentary(
        team=team,
        score=score
    )
    print(f"{i+1}. {commentary}")

Scoring Commentary Examples:
1. What a move by Kevin Durant! nails it to pull ahead for Nets.
2. Kevin Durant with the layup... Nothing but net! two points on the board for Nets.
3. Perfect feed from {assisting_player} to LeBron James makes it count! That's how it's done!
4. What a technique by Giannis Antetokounmpo! finishes to narrow the gap for Bucks.
5. What a play by LeBron James! makes it count to narrow the gap for Lakers.

Game State Commentary Examples:
1. Bucks playing excellent defense as we near the end of regulation.
2. Lakers struggling to score as we approach halftime.
3. The score is 93-79 with 3:30 remaining to play in the third quarter.


# Evaluating Generated Text Quality

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams
import re

In [ ]:
class TextEvaluator:
    def __init__(self):
        """Initialize the text evaluator with metrics."""
        # Download required NLTK resources if not already downloaded
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')

    def calculate_bleu(self, reference, candidate):
        """Calculate BLEU score between reference and candidate text."""
        # Tokenize sentences
        ref_tokens = nltk.word_tokenize(reference.lower())
        cand_tokens = nltk.word_tokenize(candidate.lower())

        # Add smoothing to handle cases with no n-gram overlaps
        smoothie = SmoothingFunction().method1

        # Calculate BLEU for different n-gram sizes
        bleu_1 = sentence_bleu([ref_tokens], cand_tokens,
                              weights=(1, 0, 0, 0), smoothing_function=smoothie)
        bleu_2 = sentence_bleu([ref_tokens], cand_tokens,
                              weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
        bleu_3 = sentence_bleu([ref_tokens], cand_tokens,
                              weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
        bleu_4 = sentence_bleu([ref_tokens], cand_tokens,
                              weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)

        return {
            "bleu-1": bleu_1,
            "bleu-2": bleu_2,
            "bleu-3": bleu_3,
            "bleu-4": bleu_4
        }

    def calculate_diversity(self, text, n=2):
        """Calculate lexical diversity using n-gram types to tokens ratio."""
        tokens = nltk.word_tokenize(text.lower())

        if len(tokens) < n:
            return 0

        n_grams = list(ngrams(tokens, n))
        unique_n_grams = set(n_grams)

        # Distinct n-grams ratio
        diversity = len(unique_n_grams) / len(n_grams)
        return diversity

    def calculate_repetition_ratio(self, text):
        """Calculate ratio of repeated words to total words."""
        tokens = nltk.word_tokenize(text.lower())
        token_count = len(tokens)

        if token_count == 0:
            return 0

        # Count occurrences of each token
        token_counts = Counter(tokens)
        repeated_tokens = sum(count - 1 for count in token_counts.values() if count > 1)

        repetition_ratio = repeated_tokens / token_count
        return repetition_ratio

    def check_factual_accuracy(self, generated_text, game_data):
        """Check if generated text matches facts from game data."""
        # This is a simplified example. In practice, you would:
        # 1. Extract entities and claims from generated text
        # 2. Compare with game data (players, scores, events)
        # 3. Calculate precision/recall of facts

        # For demonstration, we'll just check if key game entities are present
        mentioned_entities = []

        # Check for player names
        for player in game_data.get("players", []):
            if player.lower() in generated_text.lower():
                mentioned_entities.append(player)

        # Check for team names
        for team in game_data.get("teams", []):
            if team.lower() in generated_text.lower():
                mentioned_entities.append(team)

        # Calculate simple precision (% of mentioned entities that are correct)
        all_correct_entities = game_data.get("players", []) + game_data.get("teams", [])
        precision = len(mentioned_entities) / len(all_correct_entities) if all_correct_entities else 0

        return {
            "mentioned_entities": mentioned_entities,
            "precision": precision
        }

    def evaluate_commentary(self, generated_text, reference_text=None, game_data=None):
        """Comprehensive evaluation of generated commentary."""
        results = {}

        # Text-based metrics
        results["length"] = len(generated_text.split())
        results["diversity_bigram"] = self.calculate_diversity(generated_text, 2)
        results["diversity_trigram"] = self.calculate_diversity(generated_text, 3)
        results["repetition_ratio"] = self.calculate_repetition_ratio(generated_text)

        # Reference-based metrics (if reference available)
        if reference_text:
            results["bleu_scores"] = self.calculate_bleu(reference_text, generated_text)

        # Game data based metrics (if game data available)
        if game_data:
            results["factual_metrics"] = self.check_factual_accuracy(generated_text, game_data)

        return results

In [ ]:
# Demonstrate the evaluator
evaluator = TextEvaluator()

In [ ]:
# Sample data for evaluation
reference_commentary = "Curry dribbles up the court, crosses over his defender, steps back and shoots the three... BANG! That's his fifth three-pointer of the night, extending the Warriors' lead to 10 points."

In [ ]:
# Generated examples to evaluate
generated_examples = [
    # Good example - similar to reference
    "Curry brings the ball up, makes a nice move on his defender, creates space and shoots from downtown... It's good! That's another three for Curry, and the Warriors now lead by double digits.",

    # Repetitive example
    "Curry dribbles dribbles dribbles and shoots shoots shoots the ball ball ball for three three points points points. Warriors Warriors lead lead by ten ten points.",

    # Factually incorrect example
    "James drives to the basket and dunks it! The Cavaliers take the lead with that spectacular play."
]

In [ ]:
# Game data for factual evaluation
game_data = {
    "players": ["Stephen Curry", "Klay Thompson", "Draymond Green"],
    "teams": ["Warriors", "Lakers"],
    "score": {"Warriors": 78, "Lakers": 68},
    "current_player": "Stephen Curry",
    "action": "three-point shot",
    "successful": True
}

print("Commentary Evaluation Examples:")
for i, example in enumerate(generated_examples):
    print(f"\nExample {i+1}: {example}")

    # Evaluate the example
    evaluation = evaluator.evaluate_commentary(
        generated_text=example,
        reference_text=reference_commentary,
        game_data=game_data
    )

    # Display results
    print("Evaluation metrics:")
    print(f"- Length: {evaluation['length']} words")
    print(f"- Bigram diversity: {evaluation['diversity_bigram']:.3f}")
    print(f"- Trigram diversity: {evaluation['diversity_trigram']:.3f}")
    print(f"- Repetition ratio: {evaluation['repetition_ratio']:.3f} (lower is better)")

    if "bleu_scores" in evaluation:
        print(f"- BLEU-1: {evaluation['bleu_scores']['bleu-1']:.3f}")
        print(f"- BLEU-2: {evaluation['bleu_scores']['bleu-2']:.3f}")

    if "factual_metrics" in evaluation:
        print(f"- Factual precision: {evaluation['factual_metrics']['precision']:.3f}")
        print(f"- Mentioned entities: {', '.join(evaluation['factual_metrics']['mentioned_entities'])}")

    # Simple qualitative assessment
    if evaluation['repetition_ratio'] > 0.3:
        print("⚠️ This commentary is too repetitive.")

    if "factual_metrics" in evaluation and evaluation['factual_metrics']['precision'] < 0.3:
        print("⚠️ This commentary may contain factual errors.")

Commentary Evaluation Examples:

Example 1: Curry brings the ball up, makes a nice move on his defender, creates space and shoots from downtown... It's good! That's another three for Curry, and the Warriors now lead by double digits.
Evaluation metrics:
- Length: 33 words
- Bigram diversity: 1.000
- Trigram diversity: 1.000
- Repetition ratio: 0.146 (lower is better)
- BLEU-1: 0.463
- BLEU-2: 0.264
- Factual precision: 0.200
- Mentioned entities: Warriors
⚠️ This commentary may contain factual errors.

Example 2: Curry dribbles dribbles dribbles and shoots shoots shoots the ball ball ball for three three points points points. Warriors Warriors lead lead by ten ten points.
Evaluation metrics:
- Length: 26 words
- Bigram diversity: 0.815
- Trigram diversity: 1.000
- Repetition ratio: 0.500 (lower is better)
- BLEU-1: 0.250
- BLEU-2: 0.161
- Factual precision: 0.200
- Mentioned entities: Warriors
⚠️ This commentary is too repetitive.
⚠️ This commentary may contain factual errors.

Example

## Without Factual Error

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams
from collections import Counter

# Ensure punkt tokenizer is available
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

class TextEvaluator:
    def __init__(self):
        """Initialize the text evaluator with metrics."""
        pass  # Resources already checked above

    def calculate_bleu(self, reference, candidate):
        """Calculate BLEU score between reference and candidate text."""
        ref_tokens = nltk.word_tokenize(reference.lower())
        cand_tokens = nltk.word_tokenize(candidate.lower())

        smoothie = SmoothingFunction().method1

        bleu_1 = sentence_bleu([ref_tokens], cand_tokens,
                               weights=(1, 0, 0, 0), smoothing_function=smoothie)
        bleu_2 = sentence_bleu([ref_tokens], cand_tokens,
                               weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
        bleu_3 = sentence_bleu([ref_tokens], cand_tokens,
                               weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
        bleu_4 = sentence_bleu([ref_tokens], cand_tokens,
                               weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)

        return {
            "bleu-1": bleu_1,
            "bleu-2": bleu_2,
            "bleu-3": bleu_3,
            "bleu-4": bleu_4
        }

    def calculate_diversity(self, text, n=2):
        """Calculate lexical diversity using n-gram types to tokens ratio."""
        tokens = nltk.word_tokenize(text.lower())

        if len(tokens) < n:
            return 0

        n_grams = list(ngrams(tokens, n))
        unique_n_grams = set(n_grams)

        return len(unique_n_grams) / len(n_grams)

    def calculate_repetition_ratio(self, text):
        """Calculate ratio of repeated words to total words."""
        tokens = nltk.word_tokenize(text.lower())
        token_count = len(tokens)

        if token_count == 0:
            return 0

        token_counts = Counter(tokens)
        repeated_tokens = sum(count - 1 for count in token_counts.values() if count > 1)

        return repeated_tokens / token_count

    def check_factual_accuracy(self, generated_text, game_data):
        """Check if generated text matches facts from game data."""
        mentioned_entities = []

        for player in game_data.get("players", []):
            if player.lower() in generated_text.lower():
                mentioned_entities.append(player)

        for team in game_data.get("teams", []):
            if team.lower() in generated_text.lower():
                mentioned_entities.append(team)

        all_correct_entities = game_data.get("players", []) + game_data.get("teams", [])
        precision = len(mentioned_entities) / len(all_correct_entities) if all_correct_entities else 0

        return {
            "mentioned_entities": mentioned_entities,
            "precision": precision
        }

    def evaluate_commentary(self, generated_text, reference_text=None, game_data=None):
        """Comprehensive evaluation of generated commentary."""
        results = {}

        results["length"] = len(generated_text.split())
        results["diversity_bigram"] = self.calculate_diversity(generated_text, 2)
        results["diversity_trigram"] = self.calculate_diversity(generated_text, 3)
        results["repetition_ratio"] = self.calculate_repetition_ratio(generated_text)

        if reference_text:
            results["bleu_scores"] = self.calculate_bleu(reference_text, generated_text)

        if game_data:
            results["factual_metrics"] = self.check_factual_accuracy(generated_text, game_data)

        return results


# Reference text
reference_commentary = (
    "Curry dribbles up the court, crosses over his defender, steps back and shoots the three... "
    "BANG! That's his fifth three-pointer of the night, extending the Warriors' lead to 10 points."
)

# Game data
game_data = {
    "players": ["Stephen Curry", "Klay Thompson", "Draymond Green"],
    "teams": ["Warriors", "Lakers"],
    "score": {"Warriors": 78, "Lakers": 68},
    "current_player": "Stephen Curry",
    "action": "three-point shot",
    "successful": True
}

# Factually accurate generated commentary
generated_example = (
    "Stephen Curry takes control, crosses midcourt with speed, fakes the defender, "
    "steps back at the three-point line—he shoots... and it's in! "
    "That's his fifth triple tonight, putting the Warriors ahead by ten."
)

# Evaluate
evaluator = TextEvaluator()
evaluation = evaluator.evaluate_commentary(
    generated_text=generated_example,
    reference_text=reference_commentary,
    game_data=game_data
)

# Print evaluation
print("\n✅ Factually Accurate Commentary Example:")
print(f"Generated Text: {generated_example}\n")
print("Evaluation metrics:")
print(f"- Length: {evaluation['length']} words")
print(f"- Bigram diversity: {evaluation['diversity_bigram']:.3f}")
print(f"- Trigram diversity: {evaluation['diversity_trigram']:.3f}")
print(f"- Repetition ratio: {evaluation['repetition_ratio']:.3f} (lower is better)")

if "bleu_scores" in evaluation:
    print(f"- BLEU-1: {evaluation['bleu_scores']['bleu-1']:.3f}")
    print(f"- BLEU-2: {evaluation['bleu_scores']['bleu-2']:.3f}")

if "factual_metrics" in evaluation:
    print(f"- Factual precision: {evaluation['factual_metrics']['precision']:.3f}")
    print(f"- Mentioned entities: {', '.join(evaluation['factual_metrics']['mentioned_entities'])}")



✅ Factually Accurate Commentary Example:
Generated Text: Stephen Curry takes control, crosses midcourt with speed, fakes the defender, steps back at the three-point line—he shoots... and it's in! That's his fifth triple tonight, putting the Warriors ahead by ten.

Evaluation metrics:
- Length: 32 words
- Bigram diversity: 1.000
- Trigram diversity: 1.000
- Repetition ratio: 0.146 (lower is better)
- BLEU-1: 0.512
- BLEU-2: 0.339
- Factual precision: 0.400
- Mentioned entities: Stephen Curry, Warriors
